In [1]:
using BenchmarkTools
using SparseArrays
using LinearAlgebraicRepresentation
using LinearAlgebra
Lar=LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

spatial_arrangement_2(
    V::Points, 
    copEV::ChainOp, 
    copFE::ChainOp; 
    [multiproc::Bool])

##### Additional arguments:
- `multiproc::Bool`: Runs the computation in parallel mode. Defaults to `false`.

In [2]:
function spatial_arrangement_2(
    rV::Lar.Points,
    rcopEV::Lar.ChainOp,
    rcopFE::Lar.ChainOp, 
    multiproc::Bool=false)

#rcopCF = Lar.build_copFC(rV, rcopEV, rcopFE)  ######
rcopCF = Lar.Arrangement.minimal_3cycles(rV, rcopEV, rcopFE)
return rV, rcopEV, rcopFE, rcopCF
end

spatial_arrangement_2 (generic function with 2 methods)

In [3]:
V = Float64[
        0 0 0; 0 1 0;
        1 1 0; 1 0 0;
        0 0 1; 0 1 1;
        1 1 1; 1 0 1
    ]

    EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
    ])

    FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
    ])

6×12 SparseMatrixCSC{Int8, Int64} with 24 stored entries:
  1   1   1  -1   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  -1  -1   1
 -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅
  ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅
  ⋅   ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅
  ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅  -1

In [4]:
@btime spatial_arrangement_2(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),true)

  212.292 μs (3998 allocations: 250.55 KiB)


([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], sparse([1, 4, 5, 1, 2, 6, 2, 3, 7, 3  …  12, 6, 9, 10, 7, 10, 11, 8, 11, 12], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  5, 6, 6, 6, 7, 7, 7, 8, 8, 8], Int8[-1, -1, -1, 1, -1, -1, 1, -1, -1, 1  …  -1, 1, 1, -1, 1, 1, -1, 1, 1, 1], 12, 8), sparse([1, 3, 1, 4, 1, 5, 1, 6, 3, 6  …  5, 6, 2, 3, 2, 4, 2, 5, 2, 6], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  8, 8, 9, 9, 10, 10, 11, 11, 12, 12], Int8[1, -1, 1, -1, 1, -1, -1, 1, 1, -1  …  -1, 1, -1, 1, -1, 1, -1, 1, 1, -1], 6, 12), sparse([1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6], Int8[-1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1], 2, 6))

In [5]:
@code_warntype spatial_arrangement_2(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),true)

MethodInstance for spatial_arrangement_2(::Matrix{Float64}, ::SparseMatrixCSC{Int8, Int64}, ::SparseMatrixCSC{Int8, Int64}, ::Bool)
  from spatial_arrangement_2(rV::Matrix, rcopEV::SparseMatrixCSC{Int8, Int64}, rcopFE::SparseMatrixCSC{Int8, Int64}, multiproc::Bool) in Main at /Users/labaudio/Desktop/LAR-TGW-3D-master/notebooks/spatial_arrangement_2.ipynb:1
Arguments
  #self#::Core.Const(spatial_arrangement_2)
  rV::Matrix{Float64}
  rcopEV::SparseMatrixCSC{Int8, Int64}
  rcopFE::SparseMatrixCSC{Int8, Int64}
  multiproc::Bool
Locals
  rcopCF::Any
Body::Tuple{Matrix{Float64}, SparseMatrixCSC{Int8, Int64}, SparseMatrixCSC{Int8, Int64}, Any}
1 ─ %1 = Base.getproperty(Main.Lar, :Arrangement)::Any
│   %2 = Base.getproperty(%1, :minimal_3cycles)::Any
│        (rcopCF = (%2)(rV, rcopEV, rcopFE))
│   %4 = Core.tuple(rV, rcopEV, rcopFE, rcopCF)::Tuple{Matrix{Float64}, SparseMatrixCSC{Int8, Int64}, SparseMatrixCSC{Int8, Int64}, Any}
└──      return %4



# Ottimizzazione

In [6]:
function minimal_3cycles(V::Lar.Points, EV::Lar.ChainOp, FE::Lar.ChainOp)

	triangulated_faces = Array{Any, 1}(undef, FE.m)

    function face_angle(e::Int, f::Int)
        if !isassigned(triangulated_faces, f)
            @views vs_idxs = Array{Int64, 1}()
            edges_idxs = FE[f, :].nzind
            edge_num = length(edges_idxs)
            edges = zeros(Int64, edge_num, 2)

            for (i, ee) in enumerate(edges_idxs)
                edge = EV[ee, :].nzind
                edges[i, :] = edge
                vs_idxs = union(vs_idxs, edge)
            end

            #vs = V[vs_idxs, :]
			fv,edges = Lar.vcycle(EV, FE, f)

			vs = V[fv, :]


            v1 = LinearAlgebra.normalize(vs[2, :] - vs[1, :])
            v2 = [0 0 0]		# added for debug
            v3 = [0 0 0]
            err = 1e-8
            i = 3
            while -err < norm(v3) < err
                v2 = normalize(vs[i, :] - vs[1, :])
                v3 = cross(v1, v2)
                i = i + 1
            end
           M = reshape([v1; v2; v3], 3, 3)

            #vs = vs*M
			vs = (vs*M)[:, 1:2]

            # triangulated_faces[f] = Triangle.constrained_triangulation(
            #     Array{Float64,2}(vs), vs_idxs, edges, fill(true, edge_num))
			v = convert(Lar.Points, vs'[1:2,:])
			vmap = Dict(zip(fv,1:length(fv))) # vertex map
			mapv = Dict(zip(1:length(fv),fv)) # inverse vertex map
			trias = Lar.triangulate2d(v,edges)
			@views triangulated_faces[f] = [[mapv[v] for v in tria] for tria in trias]
        end
        edge_vs = EV[e, :].nzind

        t = findfirst(x->edge_vs[1] in x && edge_vs[2] in x, triangulated_faces[f])

        v1 = normalize(V[edge_vs[2], :] - V[edge_vs[1], :])

        if abs(v1[1]) > abs(v1[2])
            invlen = 1. / sqrt(v1[1]*v1[1] + v1[3]*v1[3])
            v2 = [-v1[3]*invlen, 0, v1[1]*invlen]
        else
            invlen = 1. / sqrt(v1[2]*v1[2] + v1[3]*v1[3])
            v2 = [0, -v1[3]*invlen, v1[2]*invlen]
        end

        v3 = cross(v1, v2)

        M = reshape([v1; v2; v3], 3, 3)

        triangle = triangulated_faces[f][t]
        third_v = setdiff(triangle, edge_vs)[1]
        vs = V[[edge_vs..., third_v], :]*M

        v = vs[3, :] - vs[1, :]
        angle = atan(v[2], v[3])
        return angle
    end

    #EF = FE'
    EF::SparseArrays.SparseMatrixCSC{Int8,Int} = convert(Lar.ChainOp, LinearAlgebra.transpose(FE))
	FC::SparseMatrixCSC{Int64, Int64} = Lar.Arrangement.minimal_cycles(face_angle, true)(V, EF)

	#FC'
    return -convert(Lar.ChainOp, LinearAlgebra.transpose(FC))
end



minimal_3cycles (generic function with 1 method)

In [7]:
@code_warntype minimal_3cycles(V, EV, FE)

MethodInstance for minimal_3cycles(::Matrix{Float64}, ::SparseMatrixCSC{Int8, Int64}, ::SparseMatrixCSC{Int8, Int64})
  from minimal_3cycles(V::Matrix, EV::SparseMatrixCSC{Int8, Int64}, FE::SparseMatrixCSC{Int8, Int64}) in Main at /Users/labaudio/Desktop/LAR-TGW-3D-master/notebooks/spatial_arrangement_2.ipynb:1
Arguments
  #self#::Core.Const(minimal_3cycles)
  V::Matrix{Float64}
  EV::SparseMatrixCSC{Int8, Int64}
  FE::SparseMatrixCSC{Int8, Int64}
Locals
  FC::SparseMatrixCSC{Int64, Int64}
  EF::SparseMatrixCSC{Int8, Int64}
  face_angle::var"#face_angle#12"{Matrix{Float64}, SparseMatrixCSC{Int8, Int64}, SparseMatrixCSC{Int8, Int64}, Vector{Any}}
  triangulated_faces::Vector{Any}
Body::Any
1 ─ %1  = Core.apply_type(Main.Array, Main.Any, 1)::Core.Const(Vector{Any})
│   %2  = Base.getproperty(FE, :m)::Int64
│         (triangulated_faces = (%1)(Main.undef, %2))
│   %4  = Main.:(var"#face_angle#12")::Core.Const(var"#face_angle#12")
│   %5  = Core.typeof(V)::Core.Const(Matrix{Float64})
│   %

In [8]:
function spatial_arrangement_21(
    rV::Lar.Points,
    rcopEV::Lar.ChainOp,
    rcopFE::Lar.ChainOp, 
    multiproc::Bool=false)

#rcopCF = Lar.build_copFC(rV, rcopEV, rcopFE)  ######
rcopCF::SparseArrays.SparseMatrixCSC{Int8,Int} = minimal_3cycles(rV, rcopEV, rcopFE)
return rV, rcopEV, rcopFE, rcopCF
end

spatial_arrangement_21 (generic function with 2 methods)

In [9]:
@code_warntype spatial_arrangement_21(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),true)

MethodInstance for spatial_arrangement_21(::Matrix{Float64}, ::SparseMatrixCSC{Int8, Int64}, ::SparseMatrixCSC{Int8, Int64}, ::Bool)
  from spatial_arrangement_21(rV::Matrix, rcopEV::SparseMatrixCSC{Int8, Int64}, rcopFE::SparseMatrixCSC{Int8, Int64}, multiproc::Bool) in Main at /Users/labaudio/Desktop/LAR-TGW-3D-master/notebooks/spatial_arrangement_2.ipynb:1
Arguments
  #self#::Core.Const(spatial_arrangement_21)
  rV::Matrix{Float64}
  rcopEV::SparseMatrixCSC{Int8, Int64}
  rcopFE::SparseMatrixCSC{Int8, Int64}
  multiproc::Bool
Locals
  rcopCF::SparseMatrixCSC{Int8, Int64}
Body::Tuple{Matrix{Float64}, SparseMatrixCSC{Int8, Int64}, SparseMatrixCSC{Int8, Int64}, SparseMatrixCSC{Int8, Int64}}
1 ─ %1 = Main.minimal_3cycles(rV, rcopEV, rcopFE)::Any
│   %2 = SparseArrays.SparseMatrixCSC::Core.Const(SparseMatrixCSC)
│   %3 = Core.apply_type(%2, Main.Int8, Main.Int)::Core.Const(SparseMatrixCSC{Int8, Int64})
│   %4 = Base.convert(%3, %1)::Any
│        (rcopCF = Core.typeassert(%4, %3))
│   %6 =

In [10]:
@btime spatial_arrangement_21(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),true)

  215.166 μs (3994 allocations: 250.36 KiB)


([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], sparse([1, 4, 5, 1, 2, 6, 2, 3, 7, 3  …  12, 6, 9, 10, 7, 10, 11, 8, 11, 12], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  5, 6, 6, 6, 7, 7, 7, 8, 8, 8], Int8[-1, -1, -1, 1, -1, -1, 1, -1, -1, 1  …  -1, 1, 1, -1, 1, 1, -1, 1, 1, 1], 12, 8), sparse([1, 3, 1, 4, 1, 5, 1, 6, 3, 6  …  5, 6, 2, 3, 2, 4, 2, 5, 2, 6], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  8, 8, 9, 9, 10, 10, 11, 11, 12, 12], Int8[1, -1, 1, -1, 1, -1, -1, 1, 1, -1  …  -1, 1, -1, 1, -1, 1, -1, 1, 1, -1], 6, 12), sparse([1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6], Int8[-1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1], 2, 6))